In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
 #Read in the csv from WeatherPy
city = pd.read_csv("../WeatherPy/output_data/cities.csv")
city.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kargopol,61.51,38.95,30.34,95,43,8.97,RU,1605020769
1,1,new norfolk,-42.78,147.06,64.99,61,57,2.68,AU,1605020770
2,2,airai,-8.93,125.41,63.48,100,100,1.43,TL,1605020770
3,3,velyka bilozerka,47.28,34.71,45.86,54,0,7.78,UA,1605020770
4,4,ushuaia,-54.80,-68.30,50.00,53,40,9.17,AR,1605020734


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 #use gmap, configure with api key
gmaps.configure(api_key=g_key)

In [4]:
#Create a heatmap
fig = gmaps.figure(center=(46, -5.0), zoom_level=1.5)
fig.add_layer(gmaps.heatmap_layer(city[["Lat", "Lng"]], weights=city["Humidity"], dissipating=False, max_intensity=300, point_radius=3))

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
 #My perfect vacation city will have a temperature of between 71 and 83 F, and wind speed of below 5 mph and no cloudiness at all.
new_df = city.loc[(city["Max Temp"] < 83) & (city["Wind Speed"] < 5) & (city["Max Temp"] > 71) & (city["Cloudiness"] == 0)].dropna()
new_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,calama,-22.47,-68.93,71.60,10,0,3.36,CL,1605020776
60,60,sur,22.57,59.53,78.15,72,0,3.71,OM,1605020782
223,223,vega de alatorre,20.03,-96.63,77.00,68,0,1.86,MX,1605020810
272,272,srivardhan,18.03,73.02,75.76,60,0,4.00,IN,1605020819
295,295,salalah,17.02,54.09,82.40,61,0,4.70,OM,1605020541
397,397,pihani,27.63,80.20,71.15,34,0,4.16,IN,1605020840
407,407,santa fe,-31.63,-60.70,82.99,43,0,3.00,AR,1605020831
432,432,buin,-33.73,-70.75,82.99,43,0,3.36,CL,1605020846
479,479,linares,-35.85,-71.60,74.41,43,0,4.03,CL,1605020555


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
 #hotels df has city, country, latitude and longitute
hotels = new_df[["City", "Country", "Lat", "Lng"]].copy()
hotels["Hotel Name"] = ""
hotels

,City,Country,Lat,Lng,Hotel Name
27,calama,CL,-22.47,-68.93,
60,sur,OM,22.57,59.53,
223,vega de alatorre,MX,20.03,-96.63,
272,srivardhan,IN,18.03,73.02,
295,salalah,OM,17.02,54.09,
397,pihani,IN,27.63,80.20,
407,santa fe,AR,-31.63,-60.70,
432,buin,CL,-33.73,-70.75,
479,linares,CL,-35.85,-71.60,


In [7]:
#create parameters for google maps api
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# for each of the hotels in dataframe interate through and make requests 
for i, r in hotels.iterrows():
    lat = r["Lat"]
    lng = r["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # make request
    names = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params)
    names = names.json()
    
    #save hotel name to hotels dataframe
    try:
        hotels.loc[i, "Hotel Name"] = names["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing info... skipping.")

hotels

,City,Country,Lat,Lng,Hotel Name
27,calama,CL,-22.47,-68.93,Park Hotel Calama
60,sur,OM,22.57,59.53,Sur Plaza Hotel
223,vega de alatorre,MX,20.03,-96.63,Hotel Deluxe
272,srivardhan,IN,18.03,73.02,Savli Resort/Hotel - Shrivardhan
295,salalah,OM,17.02,54.09,HAMDAN PLAZA HOTEL
397,pihani,IN,27.63,80.20,Aslam Hotel اسلم ہوٹل
407,santa fe,AR,-31.63,-60.70,Conquistador Hotel
432,buin,CL,-33.73,-70.75,Paihuen
479,linares,CL,-35.85,-71.60,Hotel Real


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels.iterrows()]
locations = hotels[["Lat", "Lng"]]

In [9]:
#On top of the heat map, put down pins for each of the ideal hotels with hotel info
fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))